In [1]:
import os
import shutil
from object_detection.ditod.Wordnn_embedding import WordnnEmbedding

/home/vinhvq11/anaconda3/envs/VGT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from torchvision import transforms
from PIL import Image

# Define the path to your image
image_path = "data/images/01_2020_TT-BTTTT_434357_page_0.jpg"

# Define the transformation to apply to the image
transform = transforms.Compose([
    transforms.ToTensor()        # Convert the image to a Torch tensor
])

# Read the image using Pillow
image = Image.open(image_path).convert('RGB')

# Apply the transformation to the image
image_tensor = transform(image)

# If you want to add batch dimension, assuming batch size is 1
image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension at the beginning

print(image_tensor.shape)  # Output shape should be [1 x 3 x H x W]

torch.Size([1, 3, 842, 596])


In [3]:
import numpy as np
text_feature = [np.load('data/grid_pkl/01_2020_TT-BTTTT_434357_page_0.pdf.pkl',allow_pickle=True)]

In [7]:
text_feature[0].keys()

dict_keys(['input_ids', 'bbox_subword_list', 'texts', 'bbox_texts_list'])

In [11]:
text_feature[0]['bbox'] = text_feature[0]['bbox_subword_list'] 

In [4]:
word_embedding = WordnnEmbedding()

use_pretrain_weight: load model from: ./object_detection/path/xlm-roberta-uncased/


In [18]:
def forward(img, batched_inputs, stride = 1):
    """ Forward computation
    Args:
        img (Tensor): in shape of [B x 3 x H x W]
        batched_inputs (list[dict]): 
    Returns:
        Tensor: in shape of [B x N x L x D], where D is the embedding_dim.
    """
    device = img.device
    batch_b, _, batch_h, batch_w = img.size()

    chargrid_map = torch.zeros((batch_b, batch_h // stride, batch_w // stride ), dtype=torch.int64).to(device)
    
    for iter_b in range(batch_b):
        per_input_ids = batched_inputs[iter_b]["input_ids"]   
        per_input_bbox = batched_inputs[iter_b]["bbox"]
        
        short_length_w = min(len(per_input_ids), len(per_input_bbox)) 
        
        if short_length_w > 0 : 
            for word_idx in range(short_length_w): 
                per_id = per_input_ids[word_idx]
                
                bbox = per_input_bbox[word_idx] / stride
                w_start, h_start, w_end, h_end = bbox.round().astype(int).tolist()
                        
                chargrid_map[iter_b, h_start:h_end, w_start: w_end] = 100
                chargrid_map[iter_b, h_start:h_end, w_start: w_end] = per_id

    return chargrid_map 

In [19]:
grid = forward(image_tensor, text_feature, stride = 1)

NameError: name 'self' is not defined

In [15]:
grid.shape

torch.Size([1, 64, 842, 596])

In [ ]:
! pip install transformers

In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained("microsoft/dit-base")


/home/vinhvq11/anaconda3/envs/VGT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BeitModel were not initialized from the model checkpoint at microsoft/dit-base and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.keys()

AttributeError: 'BeitModel' object has no attribute 'keys'

In [ ]:
import torch
torch.save(model.state_dict(), 'dit-base-224-p16-500k-62d53a.pth')

In [ ]:
model.load_state_dict()

Some weights of BeitModel were not initialized from the model checkpoint at microsoft/dit-base and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
torch.load("dit-base-224-p16-500k-62d53a.pth").keys()

odict_keys(['embeddings.cls_token', 'embeddings.mask_token', 'embeddings.position_embeddings', 'embeddings.patch_embeddings.projection.weight', 'embeddings.patch_embeddings.projection.bias', 'encoder.layer.0.lambda_1', 'encoder.layer.0.lambda_2', 'encoder.layer.0.attention.attention.query.weight', 'encoder.layer.0.attention.attention.query.bias', 'encoder.layer.0.attention.attention.key.weight', 'encoder.layer.0.attention.attention.value.weight', 'encoder.layer.0.attention.attention.value.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.layernorm_before.weight', 'encoder.layer.0.layernorm_before.bias', 'encoder.layer.0.layernorm_after.weight', 'encoder.layer.0.layernorm_after.bias', 'encoder.layer.1.lambda_1', 'encoder.layer.1.lambda_2', 'encoder.layer.1.

### Visualize Grid feature